# MongoDB Handling

After installing the MongoDB server in your machine, you can use this notebook for handling the initial processes with the database.

Specifically, in this step, we utilize Python's `pymongo` library to exploit its capabilities for MongoDB server interaction.

**Important Note: Be sure that the MongoDB server is up and running as a service in the background.**

For example, in macOS, to run MongoDB (i.e. the mongod process) as a service, run:

* `brew services start mongodb-community`

To stop a mongod running as a macOS service, use the following command as needed:

* `brew services stop mongodb-community`

To install MongoDB in your system, follow the instructions here:

* https://www.mongodb.com/docs/manual/administration/install-community/


**Note:** You can modify any of the processes below, however, you have to explain your thoughts.

In [1]:
# import library for various processes with the OS
import os

## Load configuration

In [2]:
# import library for yaml handling
import yaml

In [3]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## MongoDB database instantiation

The relevant information for the MongoDB client connection, the database name, and collection name is located in the configuration file.

```
# DB Connection with the uri (host)
client: "mongodb://localhost:27017/"

# db name
db: "aiot_course"

# db collection
col: "NAME YOUR COLLECTION"
```

In [4]:
# import library for hanlding the MongoDB client
import pymongo
# import library for retrieving datetime
from datetime import datetime

### Create the database

To create a database in MongoDB, start by creating a MongoClient object, then specify a connection URL with the correct ip address and the name of the database you want to create.

MongoDB will create the database if it does not exist, and make a connection to it.

In [5]:
client = pymongo.MongoClient(config["client"])

In [6]:
db = client[config["db"]]

### Instantiate the collection

To create a collection in MongoDB, use database object and specify the name of the collection you want to create.

MongoDB will create the collection if it does not exist.

In [7]:
col = db[config["col"]]

Initially, no collection will be shown in MongoDB before you enter the first document!

## Create the data collection

Uploading the gathered data to MongoDB collection. The data directory structure should be as follows:

```
.
└── data/
    ├── class_A/
    │   ├── data_A_01.csv
    │   ├── data_A_02.csv
    │   └── ..
    ├── class_B/
    │   ├── data_B_01.csv
    │   ├── data_B_02.csv
    │   └── .
    └── class ...
```

In [8]:
# import library for hanlding the csv data and transformations
import pandas as pd
import json

Get data path:

In [9]:
data_path = os.path.join(os.getcwd(), "data")
print(data_path)

c:\Users\User\Desktop\IoTProject\IoT-Course-AIoT-project-announcement-2025\data


List all files in a path:

In [10]:
classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print(classes_folders_list)

['class_A', 'class_D', 'class_L', 'class_M', 'class_U']


In [ ]:
# print files in folder
folder_path = os.path.join(data_path, classes_folders_list[0])
files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files_in_folder)


['data_A_01.csv', 'data_A_02.csv', 'data_A_03.csv', 'data_A_04.csv', 'data_A_05.csv', 'data_A_06.csv', 'data_A_07.csv', 'data_A_08.csv', 'data_A_09.csv', 'data_A_10.csv', 'data_A_11.csv', 'data_A_12.csv', 'data_A_13.csv', 'data_A_14.csv', 'data_A_15.csv', 'data_A_16.csv', 'data_A_17.csv', 'data_A_18.csv', 'data_A_19.csv', 'data_A_20.csv', 'data_A_21.csv', 'data_A_22.csv', 'data_A_23.csv', 'data_A_24.csv', 'data_A_25.csv', 'data_A_26.csv', 'data_A_27.csv', 'data_A_28.csv', 'data_A_29.csv', 'data_A_30.csv', 'data_A_31.csv', 'data_A_32.csv', 'data_A_33.csv', 'data_A_34.csv', 'data_A_35.csv', 'data_A_36.csv', 'data_A_37.csv', 'data_A_38.csv', 'data_A_39.csv', 'data_A_40.csv']


Each document in the MongoDB database should have the following schema:

```json
{
  "data": {
    "acc_x": ["array", "of", "values"],
    "acc_y": ["array", "of", "values"],
    "acc_z": ["array", "of", "values"],
  },
  "label": "The label of the instance",
  "datetime": "MongoDB datetime object (it can be generated with the datetime.datetime.now() function"
}
```

Accordingly, if you are using gyroscope or both accelerometer and gyroscope, the following order and naming of the sensor keys should be defined:

* for gyroscope: `gyr_x`, `gyr_y`, `gyr_z` for the three axes
* for accelerometer and gyroscope: `acc_x`, `acc_y`, `acc_z`, `gyr_x`, `gyr_y`, `gyr_z` for the six axes

**Note: Be careful, the document is mandatory to have the aforementioned schema, in order to argue and proceed with the rest of the processes later on, in data engineering, plotting, etc.**

In [12]:
from utils import df_rebase

## Provide the code to upload the data to MongoDB

In [15]:
import os
import pandas as pd
from datetime import datetime

data_root = config["data_path"]  # Π.χ. "data"

for class_dir in os.listdir(data_root):
    class_path = os.path.join(data_root, class_dir)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(class_path, filename)
                df = pd.read_csv(file_path)

                # Διάλεξε μόνο τις στήλες του γυροσκοπίου
                df = df[["x-axis (deg/s)", "y-axis (deg/s)", "z-axis (deg/s)"]]

                # Μετονομασία σε gyr_x, gyr_y, gyr_z (όπως θέλει το schema)
                df = df.rename(columns={
                    "x-axis (deg/s)": "gyr_x",
                    "y-axis (deg/s)": "gyr_y",
                    "z-axis (deg/s)": "gyr_z"
                })

                # Φτιάχνουμε το έγγραφο
                doc = {
                    "data": {col: df[col].tolist() for col in df.columns},
                    "label": class_dir,
                    "datetime": datetime.now()
                }

                # Ανεβάζουμε στο MongoDB
                collection.insert_one(doc)
                print(f"✅ Uploaded: {filename} (class: {class_dir})")


NameError: name 'collection' is not defined